# 1.导入相关库

In [1]:
import numpy as np
import xlrd
import xlwt
import copy
from scipy import optimize

# 2.基本数据读取

In [2]:
Number = 402
Week = 240
TT = 24
InitRow = 1
InitColumn = 2

Number = 402
Week = 240
InitRow = 1
InitColumn = 2

Require = np.zeros([Number , Week])
Supply = np.zeros([Number , Week])
filename = "附件1 近5年402家供应商的相关数据.xls"
file = xlrd.open_workbook(filename)
SheetName = file.sheet_names()
Sheet1 = file.sheet_by_name(sheet_name = SheetName[0])
Sheet2 = file.sheet_by_name(sheet_name = SheetName[1])
for j in range(Number):
    for i in range(Week):
        Require[j][i] = Sheet1.cell_value(rowx = j+InitRow , colx = i+InitColumn)
        Supply[j][i] = Sheet2.cell_value(rowx = j+InitRow , colx = i+InitColumn)
        pass
    pass

file2 = xlrd.open_workbook("AfterRank.xls")
SheetF = file2.sheet_by_index(4)
Fn = np.zeros(Number)
for j in range(Number):
    Fn[j] = SheetF.cell_value(rowx = j , colx = 0)
    pass

# 3.定义含有$\theta$的供应商类型Supplier2

In [3]:
class Supplier2:

    def __init__(self , s , F , kind):
        self.F  = F
        self.SetKind(kind)
        self.GetTheta(s)
        pass

    def GetTheta(self , s):
        theta = np.zeros(TT)
        for k in range(Week):
            pos = k%TT
            theta[pos] += s[k]
            pass
        theta = theta/(Week/TT)
        self.theta = theta
        pass

    def SetKind(self , kind):
        self.kind = kind
        if kind == "A":
            self.lam = 1/0.6
            self.k = 1.2
            pass
        elif kind == "B":
            self.lam = 1/0.66
            self.k = 1.1
            pass
        else:
            self.lam = 1/0.72
            self.k = 1.
            pass
        pass
    
    pass

In [4]:
ListSupplier2 = list()
Theta = np.zeros([Number , TT])
for j in range(Number):
    s = Supply[j]
    F = Fn[j]
    kind = Sheet2.cell_value(rowx = j+InitRow , colx = 1)
    item = Supplier2(s , F , kind)
    ListSupplier2.append(item)
    Theta[j] = item.theta
    pass

# 4.对24周的每周开始进行线性规划

$n$家供应商，在第$i$周供应，考虑如下情形，其中$x_{ji}$表示$j$企业在第$i$周供应量：

$$
\begin{aligned}
Z_i &= \sum_{j = 1}^n k_j x_{ji}\\
&\text{期望供应}Z_{i.\min}\\


k_j &=
\begin{cases}
1.2 , j\text{企业供应A}\\
1.1 , j\text{企业供应B}\\
1.0 , j\text{企业供应C}
\end{cases}

\end{aligned}
$$

并且对于过往五年，以24周为一个生产周期来看待，每周都有个期望供应值$\theta_{ji}$，为$j$企业在$i$周的期望供应值：

$$
\theta_{ji} = \frac{1}{10}\sum_{k=1}^{240} s_{jk}\cdot f(i , k),i = 1,2,3\cdots24\\

f(i , k) = 

\begin{cases}
1 , k\mod 24=i\\
0 , k\mod 24\neq i
\end{cases}
$$

In [5]:
Solution = list()
C = np.zeros(Number)
A = np.zeros(Number)
b = np.zeros(1)
b[0] = -2.92e4
Floor = np.zeros(Number)
for j in range(Number):
    C[j] = ListSupplier2[j].k
    A[j] = -ListSupplier2[j].lam
    pass
A = list(A)
A = np.array([A])

for i in range(TT):
    bound = list()
    for j in range(Number):
        tu = (0. , Theta[j][i])
        bound.append(tu)
        pass
    bound = tuple(bound)
    res = optimize.linprog(C , A_ub = A , b_ub = b , bounds = bound)
    Solution.append(res)

    Result = np.zeros([TT , Number])
Target = np.zeros(TT)
for i in range(TT):
    Result[i] = Solution[i].x
    Target[i] = Solution[i].fun
    pass

Result = Result.T

# 5.写出程序到excel

In [6]:
Write = xlwt.Workbook()
SheetW = Write.add_sheet("24周线性规划供货表")
for j in range(Number):
    for i in range(TT):
        SheetW.write(j , i , Result[j][i])
        pass
    pass
Write.save("24周线性规划供货表.xls")